# 0. Setup and Install AdaptNLP

In [ ]:
!pip install adaptnlp

# 1. Sequence classification fine-tuning: "bert-base-cased"

We will be loading in a pre-trained language model called "bert-base-cased" and fine-tuning it on the AG News dataset. This language model is a good base model that's been trained on a large and general set of data.


### 1.1 Model loading and taggings

In [ ]:
from adaptnlp import EasySequenceClassifier
from pprint import pprint

classifier = EasySequenceClassifier()

PyTorch version 1.6.0 available.


In [ ]:
# Inference
example_text = "This didn't work at all"

sentences = classifier.tag_text(
    text=example_text,
    model_name_or_path="bert-base-cased",
    mini_batch_size=1,
)

print("Tag Score Outputs:\n")
for sentence in sentences:
    pprint({sentence.to_original_text(): sentence.labels})

2020-09-16 17:38:13,981 loading file bert-base-cased


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Tag Score Outputs:

{"This didn't work at all": [LABEL_0 (0.3342), LABEL_1 (0.6658)]}


### 1.2. Data loading and processing with [datasets](https://github.com/huggingface/datasets)

In [ ]:
from datasets import load_dataset

train_dataset, eval_dataset = load_dataset('trec', split=['train', 'test'])

pprint(vars(train_dataset.info))

Checking /home/andrew/.cache/huggingface/datasets/d64a4904fdd2e50e1e6ea218644d57a262d44f2ca4708683474fc8acc0e92c3f.ed7f7df11daf93bb1e91f4b39dea596eada032d21abe964a4db45ebe33f18a94.py for additional imports.
Found main folder for dataset https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/trec/trec.py at /home/andrew/.cache/huggingface/modules/datasets_modules/datasets/trec
Found specific version folder for dataset https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/trec/trec.py at /home/andrew/.cache/huggingface/modules/datasets_modules/datasets/trec/ca4248481ad244f235f4cf277186cad2ee8769f975119a2bbfc41b8932b88bd7
Found script file from https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/trec/trec.py to /home/andrew/.cache/huggingface/modules/datasets_modules/datasets/trec/ca4248481ad244f235f4cf277186cad2ee8769f975119a2bbfc41b8932b88bd7/trec.py
Found dataset infos file from https://raw.githubusercontent.com/huggingface/datasets/1.0.

{'builder_name': 'trec',
 'citation': '@inproceedings{li-roth-2002-learning,\n'
             '    title = "Learning Question Classifiers",\n'
             '    author = "Li, Xin  and\n'
             '      Roth, Dan",\n'
             '    booktitle = "{COLING} 2002: The 19th International '
             'Conference on Computational Linguistics",\n'
             '    year = "2002",\n'
             '    url = "https://www.aclweb.org/anthology/C02-1150",\n'
             '}\n'
             '@inproceedings{hovy-etal-2001-toward,\n'
             '    title = "Toward Semantics-Based Answer Pinpointing",\n'
             '    author = "Hovy, Eduard  and\n'
             '      Gerber, Laurie  and\n'
             '      Hermjakob, Ulf  and\n'
             '      Lin, Chin-Yew  and\n'
             '      Ravichandran, Deepak",\n'
             '    booktitle = "Proceedings of the First International '
             'Conference on Human Language Technology Research",\n'
             '    year = "2001

In [ ]:
train_dataset.set_format(type="pandas", columns=["text", "label-coarse"])
train_dataset[:]

Set __getitem__(key) output type to pandas for ['text', 'label-coarse'] columns  (when key is int or slice) and don't output other (un-formatted) columns.


,label-coarse,text
0,0,How did serfdom develop in and then leave Russ...
1,1,What films featured the character Popeye Doyle ?
2,0,How can I find a list of celebrities ' real na...
3,1,What fowl grabs the spotlight after the Chines...
4,2,What is the full form of .com ?
...,...,...
5447,1,What 's the shape of a camel 's spine ?
5448,1,What type of currency is used in China ?
5449,4,What is the temperature today ?
5450,4,What is the temperature for cooking ?


In [ ]:
# We just run this to reformat back to a 'python' dataset
train_dataset.set_format(columns=["text", "label-coarse"])

Set __getitem__(key) output type to python objects for ['text', 'label-coarse'] columns  (when key is int or slice) and don't output other (un-formatted) columns.


### 1.3. Training

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./trec-models',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    evaluate_during_training=True,
    logging_dir='./logs',
    save_steps=100
)

In [ ]:
classifier.train(training_args=training_args,
                 train_dataset=train_dataset,
                 eval_dataset=eval_dataset,
                 model_name_or_path="bert-base-cased",
                 text_col_nm="text",
                 label_col_nm="label-coarse"
                )

Testing the mapped function outputs
Testing finished, running the mapping function on the dataset
Caching processed dataset at /home/andrew/.cache/huggingface/datasets/trec/default/1.1.0/ca4248481ad244f235f4cf277186cad2ee8769f975119a2bbfc41b8932b88bd7/cache-74eaa265e83086ed.arrow


Done writing 5452 examples in 38555508 bytes /home/andrew/.cache/huggingface/datasets/trec/default/1.1.0/ca4248481ad244f235f4cf277186cad2ee8769f975119a2bbfc41b8932b88bd7/tmptjm0pgu3.
Set __getitem__(key) output type to python objects for ['label-coarse', 'text'] columns  (when key is int or slice) and don't output other (un-formatted) columns.
Testing the mapped function outputs
Testing finished, running the mapping function on the dataset


Caching processed dataset at /home/andrew/.cache/huggingface/datasets/trec/default/1.1.0/ca4248481ad244f235f4cf277186cad2ee8769f975119a2bbfc41b8932b88bd7/cache-203ad5a27ecbd4af.arrow


Done writing 500 examples in 321995 bytes /home/andrew/.cache/huggingface/datasets/trec/default/1.1.0/ca4248481ad244f235f4cf277186cad2ee8769f975119a2bbfc41b8932b88bd7/tmpeo101k5z.
Set __getitem__(key) output type to torch for ['input_ids', 'attention_mask', 'labels'] columns  (when key is int or slice) and don't output other (un-formatted) columns.
Set __getitem__(key) output type to torch for ['input_ids', 'attention_mask', 'labels'] columns  (when key is int or slice) and don't output other (un-formatted) columns.
You are instantiating a Trainer but Tensorboard is not installed. You should consider installing it.


/home/andrew/Documents/github/adaptnlp/venv-adaptnlp/lib/python3.6/site-packages/datasets/arrow_dataset.py:835: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)
/home/andrew/Documents/github/adaptnlp/venv-adaptnlp/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{'loss': 0.959968994140625, 'learning_rate': 5e-05, 'epoch': 0.36683785766691124, 'step': 500}
{'loss': 0.388633056640625, 'learning_rate': 2.103128621089224e-05, 'epoch': 0.7336757153338225, 'step': 1000}



{'eval_loss': 0.16818927204841747, 'eval_accuracy': 0.966, 'eval_f1': array([0.97508897, 0.92222222, 0.94117647, 0.98461538, 0.97797357,
       0.96969697]), 'eval_precision': array([0.95804196, 0.96511628, 1.        , 0.98461538, 0.97368421,
       0.95238095]), 'eval_recall': array([0.99275362, 0.88297872, 0.88888889, 0.98461538, 0.98230088,
       0.98765432]), 'epoch': 0.7336757153338225, 'step': 1000}




### 1.4. Evaluation

In [ ]:
classifier.evaluate(model_name_or_path="bert-base-cased")


{'eval_loss': 0.1950705322229769, 'eval_accuracy': 0.97, 'eval_f1': array([0.97841727, 0.92391304, 1.        , 0.98461538, 0.98245614,
       0.97530864]), 'eval_precision': array([0.97142857, 0.94444444, 1.        , 0.98461538, 0.97391304,
       0.97530864]), 'eval_recall': array([0.98550725, 0.90425532, 1.        , 0.98461538, 0.99115044,
       0.97530864]), 'epoch': 1.0, 'step': 1363}


{'eval_loss': 0.1950705322229769,
 'eval_accuracy': 0.97,
 'eval_f1': array([0.97841727, 0.92391304, 1.        , 0.98461538, 0.98245614,
        0.97530864]),
 'eval_precision': array([0.97142857, 0.94444444, 1.        , 0.98461538, 0.97391304,
        0.97530864]),
 'eval_recall': array([0.98550725, 0.90425532, 1.        , 0.98461538, 0.99115044,
        0.97530864]),
 'epoch': 1.0}

In [ ]:
multiple_text = ["The countries in the northern hemisphere talked to the countries",
                 "The basketball player made a touchdown in the field goal",
                 "The market was down 40% and economists were puzzled",
                 "The engineer and the scientist made it to pluto in their rocket"]

sentences = classifier.tag_text(
    multiple_text,
    model_name_or_path="./trec-models",
    mini_batch_size=1
)

print("Tag Score Outputs:\n")
for sentence in sentences:
    pprint({sentence.to_original_text(): sentence.labels})

2020-09-16 17:40:01,844 loading file ./trec-models


/home/andrew/Documents/github/adaptnlp/venv-adaptnlp/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Predicting text: 100%|██████████| 4/4 [00:00<00:00, 130.34it/s]

Tag Score Outputs:

{'The countries in the northern hemisphere talked to the countries': [DESC (0.0008),
                                                                      ENTY (0.0006),
                                                                      ABBR (0.0001),
                                                                      HUM (0.0003),
                                                                      NUM (0.0003),
                                                                      LOC (0.9979)]}
{'The basketball player made a touchdown in the field goal': [DESC (0.0002),
                                                              ENTY (0.0007),
                                                              ABBR (0.0001),
                                                              HUM (0.9988),
                                                              NUM (0.0001),
                                                              LOC (0.0001)]}
{'The market

# 2. Sequence classification fine-tuning: "distilbert-base-uncased-finetuned-sst-2-english"


### 1.1. Model release

In [ ]:
classifier.release_model(model_name_or_path="bert-base-cased")

### 1.2. Training

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./trec-from-sst-models',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    evaluate_during_training=True,
    logging_dir='./logs',
    save_steps=100
)

In [ ]:
classifier.train(training_args=training_args,
                 train_dataset=train_dataset,
                 eval_dataset=eval_dataset,
                 model_name_or_path="distilbert-base-uncased-finetuned-sst-2-english",
                 text_col_nm="text",
                 label_col_nm="label-coarse"
                )

Testing the mapped function outputs
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Testing finished, running the mapping function on the dataset
Caching processed dataset at /home/andrew/.cache/huggingface/datasets/trec/default/1.1.0/ca4248481ad244f235f4cf277186cad2ee8769f975119a2bbfc41b8932b88bd7/cache-636eb496a50e077d.arrow


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Done writing 5452 examples in 23769660 bytes /home/andrew/.cache/huggingface/datasets/trec/default/1.1.0/ca4248481ad244f235f4cf277186cad2ee8769f975119a2bbfc41b8932b88bd7/tmpzvonbhhy.
Set __getitem__(key) output type to python objects for ['label-coarse', 'text'] columns  (when key is int or slice) and don't output other (un-formatted) columns.
Testing the mapped function outputs
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Testing finished, running the mapping function on the dataset


Caching processed dataset at /home/andrew/.cache/huggingface/datasets/trec/default/1.1.0/ca4248481ad244f235f4cf277186cad2ee8769f975119a2bbfc41b8932b88bd7/cache-caee9bfa42ee29d5.arrow


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Done writing 500 examples in 223991 bytes /home/andrew/.cache/huggingface/datasets/trec/default/1.1.0/ca4248481ad244f235f4cf277186cad2ee8769f975119a2bbfc41b8932b88bd7/tmplbpa8osr.
Set __getitem__(key) output type to torch for ['input_ids', 'attention_mask', 'labels'] columns  (when key is int or slice) and don't output other (un-formatted) columns.
Set __getitem__(key) output type to torch for ['input_ids', 'attention_mask', 'labels'] columns  (when key is int or slice) and don't output other (un-formatted) columns.
You are instantiating a Trainer but Tensorboard is not installed. You should consider installing it.


/home/andrew/Documents/github/adaptnlp/venv-adaptnlp/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{'loss': 1.0061688232421875, 'learning_rate': 5e-05, 'epoch': 0.36683785766691124, 'step': 500}
{'loss': 0.36240771484375, 'learning_rate': 2.103128621089224e-05, 'epoch': 0.7336757153338225, 'step': 1000}



{'eval_loss': 0.19626898866705597, 'eval_accuracy': 0.956, 'eval_f1': array([0.94845361, 0.9039548 , 0.8       , 0.98461538, 0.98666667,
       0.97530864]), 'eval_precision': array([0.90196078, 0.96385542, 1.        , 0.98461538, 0.99107143,
       0.97530864]), 'eval_recall': array([1.        , 0.85106383, 0.66666667, 0.98461538, 0.98230088,
       0.97530864]), 'epoch': 0.7336757153338225, 'step': 1000}




### 1.3. Evaluation

In [ ]:
classifier.evaluate(model_name_or_path="distilbert-base-uncased-finetuned-sst-2-english")


{'eval_loss': 0.15069465056993067, 'eval_accuracy': 0.972, 'eval_f1': array([0.97163121, 0.93406593, 0.875     , 0.98461538, 0.99122807,
       0.98765432]), 'eval_precision': array([0.95138889, 0.96590909, 1.        , 0.98461538, 0.9826087 ,
       0.98765432]), 'eval_recall': array([0.99275362, 0.90425532, 0.77777778, 0.98461538, 1.        ,
       0.98765432]), 'epoch': 1.0, 'step': 1363}


{'eval_loss': 0.15069465056993067,
 'eval_accuracy': 0.972,
 'eval_f1': array([0.97163121, 0.93406593, 0.875     , 0.98461538, 0.99122807,
        0.98765432]),
 'eval_precision': array([0.95138889, 0.96590909, 1.        , 0.98461538, 0.9826087 ,
        0.98765432]),
 'eval_recall': array([0.99275362, 0.90425532, 0.77777778, 0.98461538, 1.        ,
        0.98765432]),
 'epoch': 1.0}

# 3. Sequence Classification on custom language model

### 3.1. Language model fine-tuning

In [ ]:
!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-raw-v1.zip
!unzip -o wikitext-2-raw-v1.zip

train_file = "./wikitext-2-raw/wiki.train.raw"
eval_file = "./wikitext-2-raw/wiki.test.raw"

--2020-09-16 17:41:05--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-raw-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.66.22
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.66.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4721645 (4.5M) [application/zip]
Saving to: ‘wikitext-2-raw-v1.zip’

wikitext-2-raw-v1.z 100%[===================>]   4.50M  27.6MB/s    in 0.2s    

2020-09-16 17:41:06 (27.6 MB/s) - ‘wikitext-2-raw-v1.zip’ saved [4721645/4721645]

Archive:  wikitext-2-raw-v1.zip
   creating: wikitext-2-raw/
  inflating: wikitext-2-raw/wiki.test.raw  
  inflating: wikitext-2-raw/wiki.valid.raw  
  inflating: wikitext-2-raw/wiki.train.raw  


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./language-models',
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    evaluate_during_training=False,
    logging_dir='./logs',
    save_steps=2500,
    eval_steps=100
)

In [ ]:
from adaptnlp import LMFineTuner

finetuner = LMFineTuner(model_name_or_path="bert-base-cased")


/home/andrew/Documents/github/adaptnlp/venv-adaptnlp/lib/python3.6/site-packages/transformers/modeling_auto.py:821: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a Bert

In [ ]:
finetuner.train(
    training_args=training_args,
    train_file=eval_file,
    eval_file=eval_file,
    mlm=True,
    overwrite_cache=False
)

09/16/2020 17:41:09 - WARNING - adaptnlp.language_model -   Process rank: -1,
                device: cuda:0,
                n_gpu: 1,
                distributed training: False,
                16-bits training: False
            
09/16/2020 17:41:09 - INFO - adaptnlp.language_model -   Training/evaluation parameters: {
  "output_dir": "./language-models",
  "overwrite_output_dir": false,
  "do_train": false,
  "do_eval": false,
  "do_predict": false,
  "evaluate_during_training": false,
  "prediction_loss_only": false,
  "per_device_train_batch_size": 1,
  "per_device_eval_batch_size": 1,
  "per_gpu_train_batch_size": null,
  "per_gpu_eval_batch_size": null,
  "gradient_accumulation_steps": 1,
  "learning_rate": 5e-05,
  "weight_decay": 0.01,
  "adam_beta1": 0.9,
  "adam_beta2": 0.999,
  "adam_epsilon": 1e-08,
  "max_grad_norm": 1.0,
  "num_train_epochs": 1,
  "max_steps": -1,
  "warmup_steps": 500,
  "logging_dir": "./logs",
  "logging_first_step": false,
  "logging_steps": 500,
 

{'loss': 3.053148681640625, 'learning_rate': 5e-05, 'epoch': 0.9057971014492754, 'step': 500}




### 3.2. Sequence Classification task training and evaluation

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./trec-from-custom-LM-models',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    evaluate_during_training=True,
    logging_dir='./logs',
    save_steps=100
)

In [ ]:
classifier.train(training_args=training_args,
                 train_dataset=train_dataset,
                 eval_dataset=eval_dataset,
                 model_name_or_path="./language-models",
                 text_col_nm="text",
                 label_col_nm="label-coarse"
                )

Some weights of the model checkpoint at ./language-models were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./language-models and are n

Done writing 5452 examples in 38555508 bytes /home/andrew/.cache/huggingface/datasets/trec/default/1.1.0/ca4248481ad244f235f4cf277186cad2ee8769f975119a2bbfc41b8932b88bd7/tmpj9c06gac.
Set __getitem__(key) output type to python objects for ['label-coarse', 'text'] columns  (when key is int or slice) and don't output other (un-formatted) columns.
Testing the mapped function outputs
Testing finished, running the mapping function on the dataset


Caching processed dataset at /home/andrew/.cache/huggingface/datasets/trec/default/1.1.0/ca4248481ad244f235f4cf277186cad2ee8769f975119a2bbfc41b8932b88bd7/cache-56f3d2867237fb45.arrow


Done writing 500 examples in 321995 bytes /home/andrew/.cache/huggingface/datasets/trec/default/1.1.0/ca4248481ad244f235f4cf277186cad2ee8769f975119a2bbfc41b8932b88bd7/tmpxhqpsqui.
Set __getitem__(key) output type to torch for ['input_ids', 'attention_mask', 'labels'] columns  (when key is int or slice) and don't output other (un-formatted) columns.
Set __getitem__(key) output type to torch for ['input_ids', 'attention_mask', 'labels'] columns  (when key is int or slice) and don't output other (un-formatted) columns.
You are instantiating a Trainer but Tensorboard is not installed. You should consider installing it.


{'loss': 0.9473241577148438, 'learning_rate': 5e-05, 'epoch': 0.36683785766691124, 'step': 500}
{'loss': 0.45172247314453123, 'learning_rate': 2.103128621089224e-05, 'epoch': 0.7336757153338225, 'step': 1000}



{'eval_loss': 0.22290657736640423, 'eval_accuracy': 0.96, 'eval_f1': array([0.96167247, 0.92307692, 0.8       , 0.98461538, 0.97321429,
       0.97530864]), 'eval_precision': array([0.9261745 , 0.95454545, 1.        , 0.98461538, 0.98198198,
       0.97530864]), 'eval_recall': array([1.        , 0.89361702, 0.66666667, 0.98461538, 0.96460177,
       0.97530864]), 'epoch': 0.7336757153338225, 'step': 1000}




In [ ]:
classifier.evaluate(model_name_or_path="./language-models")


{'eval_loss': 0.16871526526450178, 'eval_accuracy': 0.97, 'eval_f1': array([0.97508897, 0.93478261, 0.8       , 0.98461538, 0.98678414,
       0.98159509]), 'eval_precision': array([0.95804196, 0.95555556, 1.        , 0.98461538, 0.98245614,
       0.97560976]), 'eval_recall': array([0.99275362, 0.91489362, 0.66666667, 0.98461538, 0.99115044,
       0.98765432]), 'epoch': 1.0, 'step': 1363}


{'eval_loss': 0.16871526526450178,
 'eval_accuracy': 0.97,
 'eval_f1': array([0.97508897, 0.93478261, 0.8       , 0.98461538, 0.98678414,
        0.98159509]),
 'eval_precision': array([0.95804196, 0.95555556, 1.        , 0.98461538, 0.98245614,
        0.97560976]),
 'eval_recall': array([0.99275362, 0.91489362, 0.66666667, 0.98461538, 0.99115044,
        0.98765432]),
 'epoch': 1.0}

# *Tutorials for NLP Tasks with AdaptNLP*

  1. Token Classification: NER, POS, Chunk, and Frame Tagging
      - [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Novetta/adaptnlp/blob/master/tutorials/1.%20Token%20Classification/token_tagging.ipynb)
  2. Sequence Classification: Sentiment
      - [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Novetta/adaptnlp/blob/master/tutorials/2.%20Sequence%20Classification/Easy%20Sequence%20Classifier.ipynb)
  3. Embeddings: Transformer Embeddings e.g. BERT, XLM, GPT2, XLNet, roBERTa, ALBERT
      - [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Novetta/adaptnlp/blob/master/tutorials/3.%20Embeddings/embeddings.ipynb)
  4. Question Answering: Span-based Question Answering Model
      - [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Novetta/adaptnlp/blob/master/tutorials/4.%20Question%20Answering/question_answering.ipynb)
  5. Summarization: Abstractive and Extractive
      - [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Novetta/adaptnlp/blob/master/tutorials/5.%20Summarization/summarization.ipynb)
  6. Translation: Seq2Seq
      - [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Novetta/adaptnlp/blob/master/tutorials/6.%20Translation/translation.ipynb)

### *Tutorial for Fine-tuning and Training Custom Models with AdaptNLP*

 1. Training a Sequence Classifier
   - [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Novetta/adaptnlp/blob/master/tutorials/2.%20Sequence%20Classification/Easy%20Sequence%20Classifier.ipynb)
 2. Fine-tuning a Transformers Language Model
   - [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Novetta/adaptnlp/blob/master/tutorials/Finetuning%20and%20Training%20(Advanced)/Fine-tuning%20Language%20Model.ipynb)
  
Checkout the [documentation](https://novetta.github.io/adaptnlp) for more information.
  

## *NVIDIA Docker and Configurable AdaptNLP REST Microservices*

  1. AdaptNLP official docker images are up on [Docker Hub](https://hub.docker.com/r/achangnovetta/adaptnlp).
  2. REST Microservices with AdaptNLP and FastAPI are also up on [Docker Hub](https://hub.docker.com/r/achangnovetta)
 
All images can build with GPU support if NVIDIA-Docker is correctly installed.